# Exemplo NLP

### Objetivo

### 1- Importando Libs

In [2]:
#-- importando libs
import pandas as pd
import seaborn as sns
import spacy
import re
import matplotlib.pyplot as plt

from collections import Counter
from spacy.lang.pt.stop_words import STOP_WORDS
from spacy import displacy
from spacy.tokens import Token

import pt_core_news_sm
nlp = pt_core_news_sm.load()

sns.set(rc={'figure.figsize':(15,10)}) #-- setando tamanho dos gráficos

### 2- Carregando Dataset

In [4]:
#-- abrindo o dataset das cartas
db_carta = pd.read_excel('Coleta de Dados Carta.xlsx', sheet_name='2019.1')

In [5]:
#-- printando o shape do dataset
db_carta.shape

(154, 11)

In [8]:
#-- printando % dos missing values
round(db_carta.isnull().sum().sort_values(ascending=False) / len(db_carta), 3) * 100

### 3- Trabalhando com o Dataset

In [9]:
#-- contanddo quantas vezes as palavras se repetem na coluna Querid
db_carta_Querid = db_carta['Querid'].value_counts()

In [10]:
#-- criando um Dataframe com a coluna Querid
db_carta_Querid = pd.DataFrame(db_carta_Querid)
db_carta_Querid = db_carta_Querid.reset_index()
db_carta_Querid = db_carta_Querid.head()

In [11]:
#-- conferindo a frequência de repetição das palavras
db_carta_Querid['Querid'].sort_values(ascending=False)

In [ ]:
#-- exportar o db_querido
db_carta['Querid'].to_excel('Querido_2019.xlsx')

In [12]:
#-- plotando as palavras que mais se repetem na coluna Querid
sns.barplot(x=db_carta_Querid['index'], y=db_carta_Querid['Querid'])
plt.title('Palavra que mais apareceram na coluna Querid', fontsize=25)
plt.xlabel('Palavra Preenchida', fontsize=20)
plt.ylabel(f'Coluna {db_carta_Querid.columns[1]}', fontsize=20)
plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)

### 4- Trabalhando com NLP

In [13]:
db_carta.shape[1]

11

In [14]:
#-- criando um dataset para receber as string das linhas e colunas
db_string = pd.DataFrame()

In [15]:
#-- definindo função para criação das strings
def column_string(col):
    
    """ Função para transformar as colunas de um dataset
    em uma única string"""
    
    for i in range(2, col.shape[1]):
        db_teste = pd.DataFrame()
        db_teste['teste'] = re.findall(r'\w+', str(col.iloc[:, i]))
        db_string[col.columns[i]] = db_teste['teste']

In [13]:
#-- aplicando a função column_string
column_string(db_carta)
db_string.head(30)

> Existem muitas palavras que não são interessantes de contar, por isso, irei aplicar um algoritmo para removê-las

> Verificando os stop_word da biblioteca spacy

In [18]:
#-- verificando as stop_word da biblioteca spacy
print(STOP_WORDS)

{'dezasseis', 'menor', 'muito', 'pois', 'algo', 'disso', 'demais', 'novas', 'vêm', 'dão', 'qualquer', 'nos', 'mesmo', 'próxima', 'fazes', 'mas', 'lá', 'outros', 'porque', 'na', 'possivelmente', 'pela', 'apontar', 'obrigada', 'nem', 'sei', 'foste', 'segunda', 'ou', 'dizem', 'talvez', 'põe', 'fazem', 'portanto', 'por', 'todas', 'pelas', 'sua', 'nove', 'ademais', 'como', 'adeus', 'agora', 'poder', 'estava', 'das', 'fomos', 'estará', 'onze', 'teve', 'terceira', 'você', 'põem', 'perto', 'sob', 'foi', 'comprida', 'três', 'sois', 'os', 'ambos', 'diante', 'aquela', 'doze', 'eles', 'nesta', 'meio', 'estado', 'quanto', 'vos', 'cinco', 'numa', 'depois', 'enquanto', 'parte', 'ir', 'comprido', 'parece', 'têm', 'contra', 'faço', 'cima', 'vossas', 'esses', 'muitos', 'às', 'maior', 'minhas', 'esta', 'segundo', 'tal', 'mais', 'estas', 'terceiro', 'aos', 'só', 'quer', 'fazemos', 'estes', 'bastante', 'tiveste', 'vindo', 'quem', 'meses', 'área', 'após', 'vais', 'vens', 'cuja', 'pôde', 'nuns', 'obrigado', 

In [19]:
#-- criando um objetico com as stop_words em português
stop_words_getter = lambda token: token.is_stop or token.lower_ in STOP_WORDS or token.lemma_ in STOP_WORDS
Token.set_extension('is_stop', getter=stop_words_getter)

In [20]:
#-- aplicando
for word in STOP_WORDS:
    lexeme = nlp.vocab[word]
    lexeme.is_stop = True

> Agora vou printar as palavaras presentes na coluna e identificar se são ou não uma **stopword**.

In [21]:
#-- criando um DataFrame
my_doc = pd.DataFrame()

In [23]:
#-- definindo função para criação das strings
def func_nlp(col2):
    
    """ Função para transformar as colunas de um dataset
    em uma única string"""
    
    for i in range(2, col2.shape[1]):
        db_teste = pd.DataFrame()
        db_teste['teste'] = nlp(str(col2.iloc[:, i]))
        my_doc[col2.columns[i]] = db_teste['teste']

In [24]:
#-- aplicando a função func_nlp
func_nlp(db_carta)

In [39]:
#-- printando o head
my_doc.head()

In [ ]:
#-- printando as palavras para verificar se são ou não stop_words
for i in range(0, my_doc.shape[1]):
    for token in my_doc.iloc[:,i]:
        print(token, token.lemma_, token._.is_stop)

In [27]:
#-- criando um dataframe
db_lemmas = pd.DataFrame()

In [28]:
#-- criando lista
d={}

In [14]:
#-- checando os valores
str(db_carta.iloc[:,1].values)

In [57]:
#-- aplicando a remoção das stop_words
def lemmas_remov(db_lemma):
    for i in range(2, db_carta.shape[1]):
        lemmas = [token.lemma_ for token in nlp(str(db_carta.iloc[:,i].values)) if not token.is_stop]
        lemmas = [lemma for lemma in lemmas if len(lemma)>2]
        lemmas = [lemma for lemma in lemmas if '...' not in lemma]
        lemmas = [lemma for lemma in lemmas if ' ' not in lemma]
        #db_lemmas[db_lemma.columns[i]] = lemmas
        d["string{0}".format(i)] = pd.DataFrame(lemmas)

In [15]:
#-- printando o head do dataset
my_doc.head()

In [59]:
#-- aplicando a função lemmas_remov
lemmas_remov(my_doc)

In [16]:
#d['string10']

In [85]:
teste = pd.DataFrame(d['string10'])
wordcount = Counter(teste[0])

### Exportando o dataset

In [ ]:
db_export = pd.DataFrame(list(wordcount))
db_export.head()

In [86]:
teste.to_excel(str(db_carta.columns[10])+'_2019.xlsx')

In [ ]:
#-- printando as primeiras 30 palavras que mais se repetem
wordcount.most_common(30)

### Gráficos com as palavras mais citadas

In [ ]:
#-- criando um DataFrame com o resultado da contagem
df = pd.DataFrame.from_dict(wordcount, orient='index').reset_index()

In [ ]:
#-- ordenando o dataset pela string que mais parece
df = df.sort_values(by=0, ascending=False)
df = df.head()

In [ ]:
#-- plotando as palavras que mais se repetem na coluna Estou
sns.barplot(x=df['index'], y=df[0])
plt.xlabel('Palavras mais utilizadas', fontsize=20)
plt.ylabel('Frequência das palavras', fontsize=20)
plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)
plt.title('Palavras mais utilizadas X Frequência das palavras', fontsize=25)